# Explore How To Improve Education Extraction

Idea
- split resume into 2+ parts
- make predictions on each part
- merge responses together into one JsonResume


Dev Data
- Use Resumes that QA (etc) has reported have problems with Education extraction

Goal
- Try to achieve (as close to) perfect extraction of Education on those^ problematic resumes

Post-Hoc Validation
- Collect a larg-ish collection of resumes
- Filter for those that (probably) HAVE an Education section (use regex probably)
- (A) Run Baseline: Generate JsonResume for each, compute metrics, pay attention to how much of Education section is extracted
- (B) Run Challenger: Same as above, but split each resume into two (where Education begins)
- Compare metrics of A vs B. The hypothesis to prove is that B is (much) better than A.



In [1]:
from pathlib import Path
import os
import json
import re
import base64
import json
from dataclasses import dataclass
from pathlib import Path
from nlptk import FileReader

import requests

PORT = "8001"
HOST = "10.0.0.105"
BASE_URL = f"http://{HOST}:{PORT}"
# BASE_URL = f"http://budgie.local:{PORT}"


@dataclass
class Endpoint:
    hello = f"{BASE_URL}/hello"
    static = f"{BASE_URL}/static"
    parse_resume_text = f"{BASE_URL}/parse_resume"
    parse_resume_doc = f"{BASE_URL}/parse_resume_doc"



In [2]:
pattern_dict_path = "/Users/chagerman/Data/Jobscan/Resumes/2_Annotation/LabelStudio/Resume_Sections/src/dev_data/pattern_dict.json"

data_dir = Path("/Users/chagerman/Data/Jobscan/Resumes/Problems/Missing_Education")
paths  = [data_dir.joinpath(x) for x in os.listdir(data_dir)]
paths.sort()

pattern_dict = json.loads(open(pattern_dict_path).read())

In [3]:
# response = requests.get(Endpoint.hello)
def test_parse_resume_doc(path: Path):
    filename = path.name
    with open(path, "rb") as fo:
        encoded_string = base64.b64encode(fo.read())

    payload = {"filename": filename, "filedata": encoded_string}
    response = requests.post(url=Endpoint.parse_resume_doc, data=payload)
    return response

def test_parse_resume_txt(text):
    data = {"text": text}
    headers = {"Content-Type": "application/json"}
    response = requests.post(Endpoint.parse_resume_text, headers=headers, data=json.dumps(data))
    return response


def extract_education(text, pattern_dict):
    label = "Education" 
    pattern_dict[label] == label
    education = pattern_dict[label]
    entities = []
    # pat1 = re.compile(fr"\n\s?({re.escape(term)})\b", re.IGNORECASE)
    # pat2 = re.compile(fr"\s?({re.escape(term)})\b", re.IGNORECASE)
    for term in education:
        m = re.search(fr"\n\s?({re.escape(term)})", text, re.IGNORECASE)
        if m:
            t = m.group(1)
            start, end = m.span()
            entities.append( {
                    'start': start,
                    'end': end,
                    'text': t,
                    'labels': label
                }
            )
            break
    if not entities:
        for term in education:
            m = re.search(fr"\s?({re.escape(term)})\b", text, re.IGNORECASE)
            if m:
                t = m.group(1)
                start, end = m.span()
                entities.append( {
                        'start': start,
                        'end': end,
                        'text': t,
                        'labels': label
                    }
                )
    return entities


In [4]:
iter_paths = iter(paths)
for p in paths:
    print(p.name)

Beatrice_Vicente.docx
Diana_Yousfi.pdf
Doug Gaynier - Resume 25_04_24 GM.pdf
Harleen_Singh.pdf
Jane Smith.pdf
Jon TaylorCV25_Vts.docx
Jose_Santiago.txt
Kate_Ferraro.docx
Kenzie_Tynuv.docx
Meg_Harrelson.docx
Meg_Harrelson.txt
Miguel Guerrero Quirante - Site Tester_ Operations Support_ Data Entry Specialist at Jobscan.co Resume.pdf
Resume 108.pdf
Resume 110.pdf
TMGood_resumeApril25-converse.edu.pdf


In [5]:
# path = next(iter_paths)
# path

# (A) Champion - Generate JsonResume from full text

In [6]:
path = next(iter_paths)
response = test_parse_resume_doc(path)
if response.status_code == 200:
    d = response.json()
    text = d["text"]
    jr = response.json()["jsonresume"]
    print("-"*40)
    print(path.name)
    print(f"- (A) num chars: {d['num_chars']}   num tokens:  {d['num_tokens']}")
    print(f"- (A) generation time: {d['generation_time']:.2f}")
    num_education = len(jr["education"])
    print(f"- (A) num Education: {num_education}")
    print("-"*40)
    print(json.dumps(jr, indent=2))
    # print(json.dumps(jr["education"], indent=2))
else:
    print(f"Response status code: {response.status_code}")

----------------------------------------
Beatrice_Vicente.docx
- (A) num chars: 8359   num tokens:  1987
- (A) generation time: 3.11
- (A) num Education: 0
----------------------------------------
{
  "basics": {
    "name": "BEATRICE VICENTE",
    "label": "SENIOR LEADERSHIP & OPERATIONS EXECUTIVE",
    "email": "beatrice.vicente@att.net",
    "website": "",
    "phone": "(760) 845-1160",
    "url": "http://linkedin.com/in/beatricevicente",
    "summary": "SENIOR LEADERSHIP & OPERATIONS EXECUTIVE Award-winning Senior Operations Executive with a track record of driving multi-unit retail excellence, operational efficiency, and revenue growth. Adept at P&L management, large-scale expansions, and strategic execution, leading high-performing teams to exceed financial and operational targets. Expertise in site development, process optimization, and workforce planning, delivering measurable cost savings and performance improvements. A servant leader with a passion for developing talent, fost

# (B) Challenger - Generate JsonResume from half text

In [7]:
ents = extract_education(text, pattern_dict)
print(ents)

[{'start': 8167, 'end': 8178, 'text': 'EDUCATION', 'labels': 'Education'}]


In [8]:
if ents:
    # start = ents[0]["start"]
    start = 5930
    text2 = text[start:]
    response2 = test_parse_resume_txt(text2)
    d = response2.json()
    jr2 = response2.json()["jsonresume"]
    print(f"- (B) num chars: {d['num_chars']}   num tokens:  {d['num_tokens']}")
    print(f"- (B) generation time: {d['generation_time']:.2f}")
    num_education = len(jr2["education"])
    print(f"- (B) num Education: {num_education}")
    print("-"*40)
    print()
    print(json.dumps(jr2, indent=2))

- (B) num chars: 2429   num tokens:  592
- (B) generation time: 1.80
- (B) num Education: 2
----------------------------------------

{
  "basics": {
    "name": "District Managers",
    "label": "Executive Board Member & Director",
    "email": "",
    "website": "",
    "phone": "",
    "url": "",
    "summary": "",
    "location": {},
    "profiles": []
  },
  "work": [
    {
      "name": "Homeless Veterans of San Diego",
      "position": "Executive Board Member & Director",
      "url": "",
      "location": "",
      "startDate": "2018-05-15",
      "endDate": "",
      "summary": "",
      "highlights": [
        "Board member and Director of Grant Writing, Fund Raising, and Operations for the Homeless Veterans of San Diego"
      ]
    },
    {
      "name": "San Luis Obispo Children's Museum",
      "position": "Board Member",
      "url": "",
      "location": "",
      "startDate": "2002-07-15",
      "endDate": "2005-08-15",
      "summary": "",
      "highlights": [
     

----------

## Investigate files that didn't have Education Entities

In [ ]:
paths2 = [paths[5], paths[7], paths[14]]
paths2
paths
path = paths[-1]
path

In [ ]:
reader = FileReader()
text = reader.extract_text(path)
print(text[5930:])


In [ ]:

# m = re.search("e d u c a t i o n", text, re.IGNORECASE)
# m = re.search("E D U C A T I O N", text, re.IGNORECASE)
# m

term = "e d u c a t i o n"
# term = "E D U C A T I O N"
# m = re.search(fr"\n\s?({re.escape(term)})", text, re.IGNORECASE)
# m


# pat2 = re.compile(fr"({re.escape(term)})", re.IGNORECASE)
pat2 = re.compile(fr"\s?({re.escape(term)})\b", re.IGNORECASE)
m = pat2.search(text)
m


In [ ]:
# t = text[5700:]
# m = re.search(r"\b(Education)\b", t, re.IGNORECASE)
# m

# t[m.span(1)[0]:m.span(1)[1]]

In [ ]:
# print(text[5700:])

In [ ]:
9158.87 - 7291.67 